In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import rand, seed

Define Probability Transition matrix with rows as all possible states

In [2]:
N_STATES = 2 # suppose only 2 possible states
S = np.zeros((2, 1))
P = np.zeros((2, 2))

S[0] = -1
S[1] = 1

P[0, 0] = 0.55
P[0, 1] = 1.0 - P[0, 0]
P[1, 1] = 0.55
P[1, 0] = 1.0 - P[1, 1]

In [4]:
# Display the state vector
print("State vector:\n", S)
# Display the transition matrix
print("Transition matrix:\n", P)
# Display the conditional mean vector
S_condmean = np.dot(P, S)  # expectation of St+1 if St = i
print("Conditional mean vector:\n", S_condmean)

State vector:
 [[-1.]
 [ 1.]]
Transition matrix:
 [[0.55 0.45]
 [0.45 0.55]]
Conditional mean vector:
 [[-0.1]
 [ 0.1]]


In [8]:
np.set_printoptions(precision=3, suppress=True)

Suppose that a process  𝑋𝑡
  is such that  𝑋𝑡=𝑋𝑡−1+𝑠𝑡
 , where  𝑠𝑡
  is the Markov process above. Let's simulate the process for some arbitrary parameters.

In [23]:
LEN_HIST = 50
states = np.zeros((LEN_HIST, 2), np.int8)
Xarray = np.zeros(LEN_HIST)
Psim = np.zeros((2, 2))

In [13]:
S_DICT = dict([("1",0,),
        ("2", 1),])
S_VAL = dict([("1",-1,),
        ("2", 1),])

In [17]:
# Generate sequence of uniform random numbers
randarray = rand(LEN_HIST)
# Initialize process s_0, say at state 1 (0 in Python's vector notation)
states[0, :] = (S_DICT["1"], S_VAL["1"])
Xarray[0] = 75  # states[0]

Monte-carlo exercise to simulate and find P using the actual randomely generated transition

In [25]:
for tt in range(1, LEN_HIST):
    if P[states[tt - 1, 0], states[tt - 1, 0]] > randarray[tt]:
        states[tt, :] = states[tt - 1, :]
    else:
        if states[tt - 1, 0] == S_DICT["1"]:
            states[tt, :] = [S_DICT["2"], S_VAL["2"]]
        else:
            states[tt, :] = [S_DICT["1"], S_VAL["1"]]
    Xarray[tt] = Xarray[tt - 1] + states[tt, 1]
    Psim[states[tt - 1, 0], states[tt, 0]] = (
        1.0 + Psim[states[tt - 1, 0], states[tt, 0]]
    )

In [27]:
# Compute estimated transition matrix from the Monte Carlo exercise
Pest = (Psim.T / np.sum(Psim, axis=1)).T

print(Pest)

[[0.435 0.565]
 [0.462 0.538]]


In [44]:
from numpy import linalg as LA
eigenvalues, eigenvectors = LA.eig(np.array([[0.1, 0.9], [0.25, 0.75]]))

In [45]:
eigenvalues

array([-0.15,  1.  ])

In [48]:
eigenvectors

array([[-0.964, -0.707],
       [ 0.268, -0.707]])

In [50]:
P = np.array([[0.5, 0.5], [0.1, 0.9]]) 
P40 = np.linalg.matrix_power(P, 100)
P40

array([[0.167, 0.833],
       [0.167, 0.833]])

In [49]:
# seed random number generator
seed(12345)

np.set_printoptions(precision=3, suppress=True)

P0 = np.array(
    [
        [0.9, 0.09, 0.01],
        [0.1, 0.85, 0.05],
        [0,0,1]
    ]
)

# Normalize transition matrix, ignoring NR type
P = P0[:, 0 : P0.shape[1]]  # noQA E203
P = (P.T / np.sum(P, axis=1)).T

# Verify where the ratings process converges given these estimates
P10 = np.linalg.matrix_power(P, 10)
print(P10)

P40 = np.linalg.matrix_power(P, 200)
print(P40)

# SAMPLING FROM THE MARKOV CHAIN
# Simulate how long it takes a firm to default starting with some current rating
RATINGS = dict(
    [
        ("Investment", 0),
        ("Speculative", 1),
        ("D", 2),
    ]
)
CURR_RATING = "Speculative"

N_HISTORIES = 1000
LEN_HIST = 100
histories = np.zeros((N_HISTORIES, LEN_HIST), np.int8)
histories[:, 0] = RATINGS[CURR_RATING]
randarray = rand(N_HISTORIES, LEN_HIST)

default_time = np.zeros(N_HISTORIES)
default_sum = 0

for i in range(0, N_HISTORIES):
    for j in range(1, LEN_HIST):
        for r in RATINGS:
            if randarray[i, j] < np.cumsum(P[histories[i, j - 1], :])[RATINGS[r]]:
                histories[i, j] = RATINGS[r]
                break
        if histories[i, j] == RATINGS["D"]:
            break
    # Compute the average time to default
    if np.max(histories[i, :]) == RATINGS["D"]:
        where_default = np.where((histories[i, :] == RATINGS["D"]))
        default_time[i] = where_default[0][0]
        default_sum += 1
    else:
        default_time[i] = 0.0

print("Default time:", np.sum(default_time) / default_sum)

[[0.508 0.312 0.18 ]
 [0.347 0.334 0.319]
 [0.    0.    1.   ]]
[[0.003 0.002 0.995]
 [0.002 0.002 0.996]
 [0.    0.    1.   ]]
Default time: 27.663124335812967
